
# SOLUÇÃO DA LISTA 2 - Igor Faluhelyi, 180122207


In [1]:
import pandas as pd
import numpy as np
import random

PATH = 'C:/Users/Igor/Desktop/ce2_python'

iris =  pd.read_csv(f'{PATH}/iris.data', header= None)
iris = iris.rename(columns={0:'sepal_length', 1:'sepal_width', 2:'petal_length', 3:'petal_width', 4:'class'})

def condition_class(x):
    if x == 'Iris-setosa':
        return 0
    elif x == 'Iris-versicolor':
        return 1
    else:
        return 2  
def condition_sl(x):
    if x <= 5.100000:
        return 0
    elif x <= 5.800000:
        return 1
    else:
        return 2   
def condition_sw(x):
    if x <= 2.800000:
        return 0
    elif x <= 3.000000:
        return 1
    else:
        return 2
def condition_pl(x):
    if x <= 1.600000:
        return 0
    elif x <= 4.350000:
        return 1
    else:
        return 2   
def condition_pw(x):
    if x <= 0.300000:
        return 0
    elif x <= 1.300000:
        return 1
    else:
        return 2
    
iris['class_'] = iris.iloc[:, -1].apply(condition_class)
iris['index'] = iris.index

iris['sepal_length_'] = iris.iloc[:, 0].apply(condition_sl)
iris['sepal_width_'] = iris.iloc[:, 1].apply(condition_sw)
iris['petal_length_'] = iris.iloc[:, 2].apply(condition_pl)
iris['petal_width_'] = iris.iloc[:, 3].apply(condition_pw)

iris_ = iris[['sepal_length_', 'sepal_width_', 'petal_length_', 'petal_width_', 'class_', 'index']]

iris_test_ = iris_.sample(n = 60, random_state=3, replace = False)
iris_train_ = iris_[~iris_['index'].isin(iris_.sample(n = 60, random_state=3, replace = False)['index'].values)]

## Um nó (classificação usando 1 variavel)

In [2]:
A= np.array(iris_train_.groupby('class_')['index'].agg('count'))/len(iris_train_)
B = np.log10(np.array(iris_train_.groupby('class_')['index'].agg('count'))/len(iris_train_)).transpose()

H_t = -1*(np.matmul(A, B))

In [3]:
ganho_info = []
for column in iris_train_.columns[0:-2]:
    a = iris_train_.groupby([column, 'class_'])['index'].agg(['count'])
    a = a/len(iris_train_)
    H_tc = 0
    for i in range(len(np.unique(iris_train_[column]))):
        H_tc += -1*(np.matmul(a.loc[i].values.transpose(), np.log10(a.loc[i].values.transpose()/sum(a.loc[i].values)).transpose()))
    ganho_info.append(H_t - H_tc)

n1 = iris_train_.columns[0:-2][np.argmax(ganho_info)]

d= iris_train_.groupby([n1, 'class_'])['index'].agg(['count'])/len(iris_train_)

dd = {}
for i in d.index:
    dd[i[0]] = d.loc[i[0]].index[np.argmax(d.loc[i[0]])]

def decision(x):
    if (x) in dd.keys():
        return dd[(x)]
    else:
        rng = np.random.RandomState(3)
        return rng.choice(list(dd.values()))


result1_train = list(map(decision, iris_train_[n1]))

accuracy1_train = (result1_train==iris_train_['class_'].values).sum()/len(result1_train)


result1_test = list(map(decision, iris_test_[n1]))

accuracy1_test = (result1_test==iris_test_['class_'].values).sum()/len(result1_test)

In [4]:
print(f'Acurácia no treinamento: {accuracy1_train}\n Acurácia no teste: {accuracy1_test}')

Acurácia no treinamento: 0.8222222222222222
 Acurácia no teste: 0.75


## Dois nós (classificação usando 2 variaveis)

In [5]:
A= np.array(iris_train_.groupby([n1, 'class_'])['index'].agg('count'))/len(iris_train_)
B = np.log10(np.array(iris_train_.groupby([n1, 'class_'])['index'].agg('count'))/len(iris_train_)).transpose()

H_t = -1*(np.matmul(A, B))

In [6]:
ganho_info = []
for column in pd.DataFrame(iris_train_.columns[0:-2])[~iris_train_.columns[0:-2].isin([n1])].values:
    a = iris_train_.groupby([n1, column[0], 'class_'])['index'].agg(['count'])
    a = a/len(iris_train_)
    H_tc = 0
    for i in a.index:
        H_tc += -1*(np.matmul(a.loc[i].values.transpose(), np.log10(a.loc[i].values.transpose()/sum(a.loc[i[0:-1]].values)).transpose()))
    ganho_info.append(H_t - H_tc)

n2 = pd.DataFrame(iris_train_.columns[0:-2])[~iris_train_.columns[0:-2].isin([n1])].values[np.argmax(ganho_info)][0]

d= iris_train_.groupby([n1, n2, 'class_'])['index'].agg(['count'])/len(iris_train_)

dd = {}
for i in d.index:
    #print(i[0:2], d.loc[i[0:2]].index[np.argmax(d.loc[i[0:2]])])
    dd[i[0:2]] = d.loc[i[0:2]].index[np.argmax(d.loc[i[0:2]])]

def decision(x, y):
    if (x, y) in dd.keys():
        return dd[(x, y)]
    else:
        rng = np.random.RandomState(3)
        return rng.choice(list(dd.values()))


result2_train = list(map(decision, iris_train_[n1], iris_train_[n2]))

accuracy2_train = (result2_train==iris_train_['class_'].values).sum()/len(result2_train)


result2_test = list(map(decision, iris_test_[n1], iris_test_[n2]))

accuracy2_test = (result2_test==iris_test_['class_'].values).sum()/len(result2_test)

In [7]:
print(f'Acurácia no treinamento: {accuracy2_train}\n Acurácia no teste: {accuracy2_test}')

Acurácia no treinamento: 0.8777777777777778
 Acurácia no teste: 0.8


## Três nós (classificação usando 3 variaveis)

In [8]:
A= np.array(iris_train_.groupby([n1, n2, 'class_'])['index'].agg('count'))/len(iris_train_)
B = np.log10(np.array(iris_train_.groupby([n1, n2, 'class_'])['index'].agg('count'))/len(iris_train_)).transpose()

H_t = -1*(np.matmul(A, B))

In [9]:
ganho_info = []
for column in pd.DataFrame(iris_train_.columns[0:-2])[~iris_train_.columns[0:-2].isin([n1, n2])].values:
    a = iris_train_.groupby([n1, n2, column[0], 'class_'])['index'].agg(['count'])
    a = a/len(iris_train_)
    H_tc = 0
    for i in a.index:
        H_tc += -1*(np.matmul(a.loc[i].values.transpose(), np.log10(a.loc[i].values.transpose()/sum(a.loc[i[0:-1]].values)).transpose()))
    ganho_info.append(H_t - H_tc)

n3 = pd.DataFrame(iris_train_.columns[0:-2])[~iris_train_.columns[0:-2].isin([n1, n2])].values[np.argmax(ganho_info)][0]

d= iris_train_.groupby([n1, n2, n3, 'class_'])['index'].agg(['count'])/len(iris_train_)

dd = {}
for i in d.index:
    dd[i[0:3]] = d.loc[i[0:3]].index[np.argmax(d.loc[i[0:3]])]

def decision(x, y, z):
    if (x, y, z) in dd.keys():
        return dd[(x, y, z)]
    else:
        rng = np.random.RandomState(3)
        return rng.choice(list(dd.values()))
    

result3_train = list(map(decision, iris_train_[n1], iris_train_[n2], iris_train_[n3]))

accuracy3_train = (result3_train==iris_train_['class_'].values).sum()/len(result3_train)


result3_test = list(map(decision, iris_test_[n1], iris_test_[n2], iris_test_[n3]))

accuracy3_test = (result3_test==iris_test_['class_'].values).sum()/len(result3_test)

In [10]:
print(f'Acurácia no treinamento: {accuracy3_train}\n Acurácia no teste: {accuracy3_test}')

Acurácia no treinamento: 0.8888888888888888
 Acurácia no teste: 0.8


## Quatro nós (classificação usando 4 variaveis)

In [11]:
A= np.array(iris_train_.groupby([n1, n2, n3, 'class_'])['index'].agg('count'))/len(iris_train_)
B = np.log10(np.array(iris_train_.groupby([n1, n2, n3, 'class_'])['index'].agg('count'))/len(iris_train_)).transpose()

H_t = -1*(np.matmul(A, B))

In [12]:
ganho_info = []
for column in pd.DataFrame(iris_train_.columns[0:-2])[~iris_train_.columns[0:-2].isin([n1, n2, n3])].values:
    a = iris_train_.groupby([n1, n2, n3, column[0], 'class_'])['index'].agg(['count'])
    a = a/len(iris_train_)
    H_tc = 0
    for i in a.index:
        H_tc += -1*(np.matmul(a.loc[i].values.transpose(), np.log10(a.loc[i].values.transpose()/sum(a.loc[i[0:-1]].values)).transpose()))
    ganho_info.append(H_t - H_tc)

n4 = pd.DataFrame(iris_train_.columns[0:-2])[~iris_train_.columns[0:-2].isin([n1, n2, n3])].values[np.argmax(ganho_info)][0]

d= iris_train_.groupby([n1, n2, n3, n4, 'class_'])['index'].agg(['count'])/len(iris_train_)

dd = {}
for i in d.index:
    dd[i[0:4]] = d.loc[i[0:4]].index[np.argmax(d.loc[i[0:4]])]

def decision(x, y, z, w):
    if (x, y, z, w) in dd.keys():
        return dd[(x, y, z, w)]
    else:
        rng = np.random.RandomState(3)
        return rng.choice(list(dd.values()))
    

result4_train = list(map(decision, iris_train_[n1], iris_train_[n2], iris_train_[n3], iris_train_[n4]))

accuracy4_train = (result4_train==iris_train_['class_'].values).sum()/len(result4_train)


result4_test = list(map(decision, iris_test_[n1], iris_test_[n2], iris_test_[n3], iris_train_[n4]))

accuracy4_test = (result4_test==iris_test_['class_'].values).sum()/len(result4_test)

In [13]:
print(f'Acurácia no treinamento: {accuracy4_train}\n Acurácia no teste: {accuracy4_test}')

Acurácia no treinamento: 0.9
 Acurácia no teste: 0.5666666666666667


# Considerações finais

- As melhores classificações nos dados de teste vieram das árvores de decisão com 2 e 3 nós. Apenas 1 nó não foi suficiente para distinguir o tipo da flor e com 4 nós, o algorítmo CART sofreu Overfitting, já que observamos ajuste muito bem aos dados de treinamento (90% de acerto) e péssimo resultado nos dados de teste quando comparado às outras àrvores de decisão (com menos variáveis).

- A árvore com 2 nós obteve melhores resultados, já que obteve acurácia nos dados de teste igual à arvore com 3 nós e venceu no critério de parcimônia. 